In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
import datetime
import os

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Parameters
toy = False
run_all = True
path = '/home/ubuntu/NKI/'
output_dir = '/home/ubuntu/output/'
input_dir = '/home/ubuntu/subj_info/'

n_jobs = 20 #amount of cores
cv=4
description = 'NKI_func_gridsearch_age_'
log_file = description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [3]:
log_file

'NKI_func_gridsearch_age_18-08-10-19-07-24'

In [ ]:
#len(subj)

In [5]:
# Load subj ids & ages
subj_info = pd.read_csv('/home/ubuntu/subj_info/subjnames_ages_final.csv')
#subj_subset = pd.read_csv('/home/ubuntu/subj_info/subjnames.csv')
#subj_ages = pd.read_csv('/home/ubuntu/subj_info/NKI_ages.csv')
subj = subj_info.iloc[:,0]
subj = subj.values.tolist()
y_age = subj_info.iloc[:,1]

In [6]:
len(y_age)

192

In [7]:
len(subj)

192

In [8]:
# Load volumes for each subject into a dictionary    
if run_all:
    filenames = []
    for i in range(len(subj)):
        subj[i] = str(subj[i])
        #filenames.append(path+'sub-'+subj[i]+'/func/sub-'+subj[i]+'_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz')
        filenames.append(path+subj[i]+'/session_1/BOLDrestingCAP_1/wraBOLDrestingCAP.nii.gz')
    #cofounds = []
    #for i in range(len(subj)):
        #cofounds.append(path+'sub-'+subj[i]+'/func/sub-'+subj[i]+'_task-rest_bold_confounds.tsv')



In [ ]:
#filenames

In [9]:
from nilearn import datasets

parcellations = datasets.fetch_atlas_basc_multiscale_2015(version='sym')

# We show here networks of 64, 197, 444
networks_64 = parcellations['scale064']
networks_197 = parcellations['scale197']
networks_444 = parcellations['scale444']

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure, sym_matrix_to_vec

def func_2_corr_matrix(parcellation, name):
    masker = NiftiLabelsMasker(labels_img=parcellation, standardize=True,
                               memory='nilearn_cache', verbose=0)
    #n_regions = 197
    
    correlations = []
    upper_triang_corrs = []
    # Initializing ConnectivityMeasure object with kind='correlation'
    connectome_measure = ConnectivityMeasure(kind='correlation')
    for filename in filenames:
        timeseries_each_subject = masker.fit_transform(filename)
        # call fit_transform from ConnectivityMeasure object
        correlation = connectome_measure.fit_transform([timeseries_each_subject])
        # saving each subject correlation to correlations
        correlations.append(correlation)
        # get the upper triangular correlation matrix of correlation
        upper_triang_corr = sym_matrix_to_vec(correlation, discard_diagonal=True)
        # saving each subject upper_triang_corr to upper_triang_corrs
        upper_triang_corrs.append(upper_triang_corr) 
    #   Split train test sets and save
    X_train = upper_triang_corrs[:84]
    X_test = upper_triang_corrs[84:]
    y_train = y_age[:84]
    y_test = y_age[84:]
    random_subj_train = subj[:84]
    random_subj_test = subj[84:]
    np.savez_compressed(output_dir+log_file+'NKI_func'+name+'_train_set',a=X_train, b=y_train, c=random_subj_train)
    np.savez_compressed(output_dir+log_file+'NKI_func'+name+'_test_set',a=X_test, b=y_test, c=random_subj_test)
    return correlations, upper_triang_corrs

correlations197, upper_triang_corrs197 = func_2_corr_matrix(networks_197, '197')
correlations444, upper_triang_corrs444 = func_2_corr_matrix(networks_444, '444')

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
if not run_all:
    loaded = np.load(output_dir+'train_set.npz')
    X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']

In [ ]:
loaded = np.load(output_dir+'train_set.npz')
X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']